In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [2]:
data=pd.read_csv("Fraud_check.csv")

In [3]:
data=data.rename({'Marital.Status':'MaritalStatus','Taxable.Income':'TaxableIncome','City.Population':'CityPopulation','Work.Experience':'WorkExperience'},axis=1)

In [4]:
data

,Undergrad,MaritalStatus,TaxableIncome,CityPopulation,WorkExperience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


In [5]:
A=data["TaxableIncome"].copy()
A[data['TaxableIncome']<30000]=1
A[data['TaxableIncome']>=30000]=0
data["TaxableIncome"]=A.copy()

In [6]:
A=preprocessing.LabelEncoder()
data['Undergrad']=A.fit_transform(data['Undergrad'])
data['Urban']=A.fit_transform(data['Urban'])
data['MaritalStatus']=A.fit_transform(data['MaritalStatus'])

In [7]:
data

,Undergrad,MaritalStatus,TaxableIncome,CityPopulation,WorkExperience,Urban
0,0,2,0,50047,10,1
1,1,0,0,134075,18,1
2,0,1,0,160205,30,1
3,1,2,0,193264,15,1
4,0,1,0,27533,28,0
...,...,...,...,...,...,...
595,1,0,0,39492,7,1
596,1,0,0,55369,2,1
597,0,0,0,154058,0,1
598,1,1,0,180083,17,0


In [8]:
data.groupby(['TaxableIncome']).count()

,Undergrad,MaritalStatus,CityPopulation,WorkExperience,Urban
TaxableIncome,,,,,
0,476,476,476,476,476
1,124,124,124,124,124


In [9]:
476/(124+476) #if i assign 0 to all resultant variable i will have a accuracy of  

0.7933333333333333

In [10]:
data.columns

Index(['Undergrad', 'MaritalStatus', 'TaxableIncome', 'CityPopulation',
       'WorkExperience', 'Urban'],
      dtype='object')

In [11]:
data=data[['TaxableIncome','Undergrad','MaritalStatus','CityPopulation','WorkExperience','Urban']]
data

,TaxableIncome,Undergrad,MaritalStatus,CityPopulation,WorkExperience,Urban
0,0,0,2,50047,10,1
1,0,1,0,134075,18,1
2,0,0,1,160205,30,1
3,0,1,2,193264,15,1
4,0,0,1,27533,28,0
...,...,...,...,...,...,...
595,0,1,0,39492,7,1
596,0,1,0,55369,2,1
597,0,0,0,154058,0,1
598,0,1,1,180083,17,0


In [12]:
X=np.array(data.iloc[:,1:])
Y=np.array(data.iloc[:,0:1]).ravel()

In [13]:
X_train, X_test, Y_train, Y_test= train_test_split(X,Y,test_size=0.2,random_state=42,stratify=Y)
X_train.shape,Y_train.shape

((480, 5), (480,))

In [20]:
parameters={'n_estimators':[75,100,200,300],'max_features':[1,2,3,4,5]}
model=RandomForestClassifier(random_state=230)
model=GridSearchCV(model,parameters,cv=5,scoring='accuracy')
model.fit(X_train,Y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=230),
             param_grid={'max_features': [1, 2, 3, 4, 5],
                         'n_estimators': [75, 100, 200, 300]},
             scoring='accuracy')

In [21]:
model.best_params_

{'max_features': 1, 'n_estimators': 300}

In [31]:
num_tree=100
max_feature=1
Kfold= KFold(n_splits=10,random_state=230)
model=RandomForestClassifier(n_estimators=300,max_features=max_feature)
results=cross_val_score(model,X_train,Y_train,cv=Kfold)
print(results.mean())

C:\Users\MRITYUNJAY\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


0.7416666666666667


In [32]:
model.fit(X_train,Y_train)

RandomForestClassifier(max_features=1, n_estimators=300)

In [33]:
Y_pred=model.predict(X_test)

In [34]:
from sklearn.metrics import confusion_matrix
confusion_matrix=confusion_matrix(Y_pred, Y_test)
print(confusion_matrix)

[[88 25]
 [ 7  0]]


In [35]:
(confusion_matrix[0,0]+confusion_matrix[1,1])/confusion_matrix.sum()

0.7333333333333333